update_time 으로 끊기

pid uid는 시간이랑 상관이없다,,

앱 상에서 데이터 가져오는거는 2차데이터 필요없음

location, name, pid , price , product_image, tag, updatetime , status

In [46]:
import requests
import json
import pandas as pd
from datetime import datetime

def bunjang(key_word, pages):
    pid = []
    for page in range(pages):
        url = f'https://api.bunjang.co.kr/api/1/find_v2.json?order=date&n=200&page={page}&req_ref=search&q={key_word}&stat_device=w&stat_category_required=1&version=4'
        response = requests.get(url)
        datas = response.json()['list']
        ids = [data['pid'] for data in datas]
        pid.extend(ids)
        items=[]
        for id in pid:
            url = f'https://api.bunjang.co.kr/api/1/product/{id}/detail_info.json?version=4'
            response = requests.get(url)
            try:
                details = response.json()['item_info']
                details.pop('category_name')
                details.pop('pay_option')
                items.append(details)
            except:
                print('error')
        df = pd.DataFrame(items)
        bunjang_df = df[['name','price','location','description_for_detail','num_item_view','pid']]
        
        bunjang_df['url'] = 'https://m.bunjang.co.kr/products/'+ bunjang_df['pid']
        bunjang_df['market'] = '번개장터'
        bunjang_df['keyword'] = key_word
#         bunjang_df.drop(['pid'], axis=1)
        
        return bunjang_df
    


In [47]:
categories = ["자전거"]
a=[]
for category in categories:
    a.append(bunjang(category,5))

<ipython-input-46-d1277c854409>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bunjang_df['url'] = 'https://m.bunjang.co.kr/products/'+ bunjang_df['pid']
<ipython-input-46-d1277c854409>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bunjang_df['market'] = '번개장터'
<ipython-input-46-d1277c854409>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [48]:
a[0]

,name,price,location,description_for_detail,num_item_view,pid,url,market,keyword
0,자이언트 tcr 프로팀 썬웹 올라운드 풀카본 로드 자전거,2500000,서울특별시 동대문구 청량리동,판매 물품: 자이언트 tcr 프로팀 썬웹 \n\n년식: 2018\n\n마일리지: 얼...,0,186206050,https://m.bunjang.co.kr/products/186206050,번개장터,자전거
1,로드 자전거 기함급,2150000,경기도 평택시 비전2동,안녕하세요 ^^ 로드 자전거 판매 합니다.\n\n에이모션사에 트리가 레드 입니다.\...,1780,176233979,https://m.bunjang.co.kr/products/176233979,번개장터,자전거
2,자전거 브레이크 스램 포스,140000,경기도 평택시 비전2동,안녕하세요 ^^ 미사용 스램 포스 브레이크 팝니다~\n\n저는 다이렉트 브레이크가 ...,107,181992009,https://m.bunjang.co.kr/products/181992009,번개장터,자전거
3,실내 자전거 스피닝 전문가용자전거,350000,인천광역시 부평구 부평3동,"2019년 100만원구입\n완전A+++급\n인천지역 ,근교,직접 배송",143,182666035,https://m.bunjang.co.kr/products/182666035,번개장터,자전거
4,9LED 자전거 안전등,15000,서울특별시 강서구 화곡제3동,9LED 자전거 안전등 총3개 전부 일괄 판매합니다.\n\n미개봉 새제품입니다. ...,0,185893784,https://m.bunjang.co.kr/products/185893784,번개장터,자전거
...,...,...,...,...,...,...,...,...,...
195,라이노워크. 자전거핸들바 가방,24900,서울특별시 동작구 상도제1동,새상품 스포츠 등산 가방,0,186190853,https://m.bunjang.co.kr/products/186190853,번개장터,자전거
196,휴대용 자전거펌프,15000,전라북도 익산시 모현동,휴대용 자전거 펌프 입니다\nco2기능도 있어요,11,185876692,https://m.bunjang.co.kr/products/185876692,번개장터,자전거
197,자전거져지 사이즈S 프로팀,20000,전라북도 익산시 모현동,네이비 하늘색 검정색 져지입니다\n프로팀재질 슬림핏입니다\n각 1장당 2만원입니다\...,130,183631289,https://m.bunjang.co.kr/products/183631289,번개장터,자전거
198,YKYWBIKE 화이트져지 사이즈XS,20000,전라북도 익산시 모현동,화이트져지입니다\n프로팀재질 슬림핏입니다\n한번도 안입은 새상품입니다\n사이즈XS\n,144,183627983,https://m.bunjang.co.kr/products/183627983,번개장터,자전거


In [50]:
page=0
key_word="커피 믹스"
url = f'https://api.bunjang.co.kr/api/1/find_v2.json?order=date&n=96&page={page}&req_ref=search&q={key_word}&stat_device=w&stat_category_required=1&version=4'
data = requests.get(url).json()['list']

In [51]:
df=pd.DataFrame(data)
df

,ad,bizseller,checkout,contact_hope,free_shipping,is_adult,is_super_up_shop,location,max_cpc,name,...,super_up,tag,uid,update_time,used,bun_pay_filter_enabled,inspection,imp_id,ad_ref,faved
0,False,False,False,False,False,False,None,서울특별시 서초구 잠원동,None,맥심 믹스커피250개 유효기간2023년9월,...,None,,5995015,1650968960,1,False,None,31386267cae5263c2c98,,False
1,False,False,False,False,True,False,None,,None,프렌치카페 300스틱 무료배송 (커피믹스),...,None,프렌치카페 커피믹스,77094208,1650968609,1,False,None,31386267cae5263c2c99,,False
2,False,False,False,False,True,False,None,,None,420개(무료배송) 프렌치카페 커피믹스- 싸고 맛있는 믹스커피,...,None,커피믹스 프렌치카페 남양 400개,6288370,1650966043,2,True,None,31386267cae5263c2c9a,,False
3,False,False,False,False,True,False,None,서울특별시 동작구 대방동,None,프렌치카페 300티 커피믹스 무료배송!,...,None,커피 스타벅스 커피빈 맥심 모카,6776422,1650964440,2,False,None,31386267cae5263c2c9b,,False
4,False,False,False,False,False,False,None,,None,암웨이 아라비카 골드 커피믹스 100p,...,None,암웨이커피 커피믹스,79120724,1650964390,2,True,None,31386267cae5263c2c9c,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,False,False,False,False,True,False,None,,None,할리스 바닐라딜라이트 커피 스틱 분말 믹스,...,None,맥심 커피믹스 바닐라라떼 바닐라빈 홈카페,79235033,1650186592,2,True,None,31386267cae5263c2cf3,,False
92,False,False,False,False,True,False,None,,None,커피믹스 400T,...,None,번개톡,11418058,1650171895,1,False,None,31386267cae5263c2cf4,,False
93,False,False,False,False,True,False,None,,None,(무배)맥심 모카골드 커피믹스 180T+180T,...,None,맥심 커피믹스,4257355,1650159350,2,True,None,31386267cae5263c2cf5,,False
94,False,False,False,False,True,False,None,부산광역시 사하구 하단제2동,None,"이미 유명한 당제로 커피믹스, 여기 구해왔습니다 !",...,None,당뇨커피 엄마선물 아빠선물,77922508,1650156988,2,True,None,31386267cae5263c2cf6,,False


In [57]:
col =['location', 'name',
       'num_comment', 'num_faved', 'only_neighborhood', 'pid',
       'price', 'product_image', 'status','tag', 'uid', 'update_time', 'used','imp_id']
df=df[col]

In [61]:
df.sort_values('imp_id')

,location,name,num_comment,num_faved,only_neighborhood,pid,price,product_image,status,tag,uid,update_time,used,imp_id
0,서울특별시 서초구 잠원동,맥심 믹스커피250개 유효기간2023년9월,0,0,False,186205820,23000,https://media.bunjang.co.kr/product/186205820_...,0,,5995015,1650968960,1,31386267cae5263c2c98
1,,프렌치카페 300스틱 무료배송 (커피믹스),0,0,False,186205212,25000,https://media.bunjang.co.kr/product/186205212_...,0,프렌치카페 커피믹스,77094208,1650968609,1,31386267cae5263c2c99
2,,420개(무료배송) 프렌치카페 커피믹스- 싸고 맛있는 믹스커피,0,28,False,170431189,95700,https://media.bunjang.co.kr/product/170431189_...,0,커피믹스 프렌치카페 남양 400개,6288370,1650966043,2,31386267cae5263c2c9a
3,서울특별시 동작구 대방동,프렌치카페 300티 커피믹스 무료배송!,0,9,False,93321602,27800,https://media.bunjang.co.kr/product/93321602_1...,0,커피 스타벅스 커피빈 맥심 모카,6776422,1650964440,2,31386267cae5263c2c9b
4,,암웨이 아라비카 골드 커피믹스 100p,0,0,False,184960375,17000,https://media.bunjang.co.kr/product/184960375_...,0,암웨이커피 커피믹스,79120724,1650964390,2,31386267cae5263c2c9c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,,할리스 바닐라딜라이트 커피 스틱 분말 믹스,0,1,False,185358978,14000,https://media.bunjang.co.kr/product/185358978_...,0,맥심 커피믹스 바닐라라떼 바닐라빈 홈카페,79235033,1650186592,2,31386267cae5263c2cf3
92,,커피믹스 400T,0,0,False,185334651,41000,https://media.bunjang.co.kr/product/185334651_...,0,번개톡,11418058,1650171895,1,31386267cae5263c2cf4
93,,(무배)맥심 모카골드 커피믹스 180T+180T,0,2,False,185317957,39800,https://media.bunjang.co.kr/product/185317957_...,0,맥심 커피믹스,4257355,1650159350,2,31386267cae5263c2cf5
94,부산광역시 사하구 하단제2동,"이미 유명한 당제로 커피믹스, 여기 구해왔습니다 !",0,2,False,180925536,17900,https://media.bunjang.co.kr/product/180925536_...,0,당뇨커피 엄마선물 아빠선물,77922508,1650156988,2,31386267cae5263c2cf6


In [59]:
df.sort_values('update_time')

,location,name,num_comment,num_faved,only_neighborhood,pid,price,product_image,status,tag,uid,update_time,used,imp_id
95,인천광역시 남동구 간석4동,스테비아커피믹스( 무당) 판매~,0,8,False,175600413,50000,https://media.bunjang.co.kr/product/175600413_...,0,스테비아커피 무당커피 당뇨,13019229,1650098913,1,31386267cae5263c2cf7
94,부산광역시 사하구 하단제2동,"이미 유명한 당제로 커피믹스, 여기 구해왔습니다 !",0,2,False,180925536,17900,https://media.bunjang.co.kr/product/180925536_...,0,당뇨커피 엄마선물 아빠선물,77922508,1650156988,2,31386267cae5263c2cf6
93,,(무배)맥심 모카골드 커피믹스 180T+180T,0,2,False,185317957,39800,https://media.bunjang.co.kr/product/185317957_...,0,맥심 커피믹스,4257355,1650159350,2,31386267cae5263c2cf5
92,,커피믹스 400T,0,0,False,185334651,41000,https://media.bunjang.co.kr/product/185334651_...,0,번개톡,11418058,1650171895,1,31386267cae5263c2cf4
91,,할리스 바닐라딜라이트 커피 스틱 분말 믹스,0,1,False,185358978,14000,https://media.bunjang.co.kr/product/185358978_...,0,맥심 커피믹스 바닐라라떼 바닐라빈 홈카페,79235033,1650186592,2,31386267cae5263c2cf3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,,암웨이 아라비카 골드 커피믹스 100p,0,0,False,184960375,17000,https://media.bunjang.co.kr/product/184960375_...,0,암웨이커피 커피믹스,79120724,1650964390,2,31386267cae5263c2c9c
3,서울특별시 동작구 대방동,프렌치카페 300티 커피믹스 무료배송!,0,9,False,93321602,27800,https://media.bunjang.co.kr/product/93321602_1...,0,커피 스타벅스 커피빈 맥심 모카,6776422,1650964440,2,31386267cae5263c2c9b
2,,420개(무료배송) 프렌치카페 커피믹스- 싸고 맛있는 믹스커피,0,28,False,170431189,95700,https://media.bunjang.co.kr/product/170431189_...,0,커피믹스 프렌치카페 남양 400개,6288370,1650966043,2,31386267cae5263c2c9a
1,,프렌치카페 300스틱 무료배송 (커피믹스),0,0,False,186205212,25000,https://media.bunjang.co.kr/product/186205212_...,0,프렌치카페 커피믹스,77094208,1650968609,1,31386267cae5263c2c99


In [56]:
df['idx']=range(len(df))
df.sort_values('pid')

,ad,bizseller,checkout,contact_hope,free_shipping,is_adult,is_super_up_shop,location,max_cpc,name,...,tag,uid,update_time,used,bun_pay_filter_enabled,inspection,imp_id,ad_ref,faved,idx
42,False,False,False,False,True,False,None,,None,암웨이카페드다몬커피믹스(100스틱),...,암웨이 암웨이커피 식품,6494609,1650861181,2,False,None,31386267cae5263c2cc2,,False,42
82,False,False,False,False,True,False,None,경기도 부천시 중3동,None,맥심 카누 마일드 로스트 아메리카노 미니 원두믹스 30개입 운포 8000,...,커피 아메리카노 원두 라떼 커플,2568754,1650361520,2,True,None,31386267cae5263c2cea,,False,82
84,False,False,False,False,True,False,None,경기도 의정부시 의정부2동,None,G7 3in1 커피믹스 16g 100개 무료배송 g7 믹스커피,...,g7 G7 믹스커피 3in1 커피믹스,5951350,1650341473,2,False,None,31386267cae5263c2cec,,False,84
41,False,False,False,False,False,False,None,경기도 용인시 기흥구 구성동,None,(새상품) SL 커피믹스,...,커피 믹스커피 선물 커피믹스240개 커피원두,6102631,1650865697,2,False,None,31386267cae5263c2cc1,,False,41
14,False,False,False,False,True,False,None,대구광역시 남구 봉덕1동,None,미사용)맥심 홈카페 키트 + 믹스커피,...,홈카페 맥심 우드트레이 머그잔 머그컵,3457994,1650940733,2,True,None,31386267cae5263c2ca6,,False,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,False,False,False,False,True,False,None,,None,프렌치카페 300스틱 무료배송 (커피믹스),...,프렌치카페 커피믹스,77094208,1650968609,1,False,None,31386267cae5263c2c99,,False,1
0,False,False,False,False,False,False,None,서울특별시 서초구 잠원동,None,맥심 믹스커피250개 유효기간2023년9월,...,,5995015,1650968960,1,False,None,31386267cae5263c2c98,,False,0
38,False,False,False,False,True,False,None,대방역(수도권1호선),None,프렌치카페 300티 커피믹스 무료배송!,...,커피믹스,5532325,1650877240,2,False,None,31386267cae5263c2cbe,,False,38
5,False,False,False,False,True,False,None,서울특별시 동작구 대방동,None,프렌치카페 300티 커피믹스 무료배송!,...,,3314862,1650964075,2,False,None,31386267cae5263c2c9d,,False,5


In [53]:
df.columns

Index(['ad', 'bizseller', 'checkout', 'contact_hope', 'free_shipping',
       'is_adult', 'is_super_up_shop', 'location', 'max_cpc', 'name',
       'num_comment', 'num_faved', 'only_neighborhood', 'outlink_url', 'pid',
       'price', 'product_image', 'pu_id', 'ref_campaign', 'ref_code',
       'ref_medium', 'ref_content', 'ref_source', 'ref_test', 'status',
       'style', 'super_up', 'tag', 'uid', 'update_time', 'used',
       'bun_pay_filter_enabled', 'inspection', 'imp_id', 'ad_ref', 'faved'],
      dtype='object')

In [36]:
df['used'].unique()

array([2, 1], dtype=int64)

In [30]:
page=2
key_word="커피 믹스"
url = f'https://api.bunjang.co.kr/api/1/find_v2.json?order=date&n=100&page={page}&req_ref=search&q={key_word}&stat_device=w&stat_category_required=1&version=4'
response = requests.get(url)
datas = response.json()['list']
ids = [data['pid'] for data in datas]
items=[]

In [32]:
datas[2]

{'ad': False,
 'bizseller': False,
 'checkout': False,
 'contact_hope': False,
 'free_shipping': True,
 'is_adult': False,
 'is_super_up_shop': None,
 'location': '충청북도 청주시 청원구 내수읍',
 'max_cpc': None,
 'name': '에소프레소잔 커피잔/믹스잔',
 'num_comment': '0',
 'num_faved': '0',
 'only_neighborhood': False,
 'outlink_url': '',
 'pid': '139796651',
 'price': '33000',
 'product_image': 'https://media.bunjang.co.kr/product/139796651_1_1631949132_w{res}.jpg',
 'pu_id': None,
 'ref_campaign': None,
 'ref_code': None,
 'ref_medium': None,
 'ref_content': '',
 'ref_source': '',
 'ref_test': None,
 'status': '0',
 'style': '',
 'super_up': None,
 'tag': '커피잔 에소프레소잔 커피믹스잔 후식차컵 소스컵',
 'uid': '12683129',
 'update_time': 1639104183,
 'used': 1,
 'bun_pay_filter_enabled': False,
 'inspection': None,
 'imp_id': '3edf6268f36039c24fcf',
 'ad_ref': '',
 'faved': False}

In [71]:
set(ids_200).intersection(set(ids_96))

set()

In [66]:
 "178181290" in ids_200

False

In [63]:
ids_96

['185214771',
 '173022982',
 '178181290',
 '180984489',
 '123737937',
 '184181433',
 '184976760',
 '184976324',
 '184970104',
 '184819090',
 '180922835',
 '178472725',
 '184932101',
 '183597241',
 '167680588',
 '180432310',
 '175562768',
 '180449598',
 '184787777',
 '179854741',
 '183284480',
 '184677717',
 '131373981',
 '136395025',
 '160462916',
 '174781665',
 '184473666',
 '142116890',
 '184410699',
 '184322943',
 '163876831',
 '163877594',
 '184253712',
 '184169548',
 '184168982',
 '183923577',
 '183960756',
 '183778232',
 '183646223',
 '178985993',
 '150456707',
 '148346308',
 '149495662',
 '129837853',
 '136090868',
 '135527066',
 '71119547',
 '181817988',
 '182554114',
 '182488502',
 '182368008',
 '181925593',
 '172428052',
 '148935633',
 '182055192',
 '153422346',
 '181744534',
 '181744408',
 '181376054',
 '173802775',
 '124399292',
 '165375211',
 '180889790',
 '180849482',
 '180538014',
 '179218419',
 '178473373',
 '178847948',
 '168864472',
 '168864058',
 '172645627',
 '17939

In [14]:
import requests
id=178181290
url = f'https://api.bunjang.co.kr/api/1/product/{id}/detail_info.json?version=4'
response = requests.get(url)

details = response.json()
#     details.pop('category_name')
#     details.pop('pay_option')

In [16]:
details

{'item_info': {'pid': '178181290',
  'uid': '8234137',
  'name': '텀블러 회오리 스키니텀블러 커피 쏘맥 믹스텀블러',
  'status': '0',
  'price': '6000',
  'num_faved': '1',
  'num_item_view': '36',
  'num_comment': '0',
  'user_name': '뉴시그널',
  'update_time': 1649976931.0,
  'free_shipping': False,
  'profile_image': 'https://media.bunjang.co.kr/images/crop/645207713_w{res}.jpg',
  'product_image': 'https://media.bunjang.co.kr/product/178181290_{cnt}_1644918648_w{res}.jpg',
  'bizseller': False,
  'category_id': '800400040',
  'category_name': [{'800': '생활/가공식품'},
   {'800400': '주방용품'},
   {'800400040': '텀블러/물병'}],
  'description': '스키니텀블러 \n원터치 회오리 자동 믹스 텀블러\n커피 미숫가루 쏘맥 썩을때 소용돌이를 만들어 \n섞는 텀블러 입니다.\n새 상품입니다 \n직거래 6000원 \n수원 율전동 입니다 .\n쳇주세요 ',
  'description_for_detail': '스키니텀블러 \n원터치 회오리 자동 믹스 텀블러\n커피 미숫가루 쏘맥 썩을때 소용돌이를 만들어 \n섞는 텀블러 입니다.\n새 상품입니다 \n직거래 6000원 \n수원 율전동 입니다 .\n쳇주세요 ',
  'tradable': False,
  'used': True,
  'used_code': 1,
  'image_count': 5,
  'image_file_list_for_edit': [],
  'image_source': '